In [1]:
# Install eli5, you can use any other model Interpretability libraries
!pip install eli5 -q

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import eli5
import warnings
warnings.filterwarnings('ignore')

2023-03-30 18:35:44.118522: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-30 18:35:50.270848: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-30 18:35:50.270956: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-30 18:35:50.939301: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-03-30 18:36:03.057440: W tensorflow/stream_executor/platform/de

In [3]:
# Load files
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')
ss = pd.read_csv('SampleSubmission.csv')

In [4]:
# Preview train
train.head()

,child_id,data_year,child_date,child_age,child_enrolment_date,child_months_enrolment,child_grant,child_years_in_programme,child_height,child_observe_attentive,...,obs_cooking_5,obs_cooking_6,obs_heating_1,obs_heating_2,obs_heating_3,obs_heating_4,obs_heating_5,obs_heating_6,obs_heating_7,target
0,ID_SYSJ2FM0D,2022.0,2022-02-03,59.000000,NaN,NaN,NaN,NaN,NaN,Sometimes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,51.500000
1,ID_J5BTFOZR3,2019.0,NaN,60.163933,NaN,NaN,NaN,1st year in the programme,103.000000,Sometimes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55.869999
2,ID_R00SN7AUD,2022.0,2022-03-11,69.000000,NaN,NaN,NaN,NaN,108.400002,Often,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.520000
3,ID_BSSK60PAZ,2021.0,2021-10-13,53.000000,2020-01-15,20.0,No,1st year in the programme,98.099998,Almost always,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58.599998
4,ID_IZTY6TC4D,2021.0,2021-10-13,57.000000,2021-10-13,0.0,NaN,2nd year in programme,114.000000,Almost always,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,76.599998


In [5]:
# Preview test
test.head()

,child_id,data_year,child_date,child_age,child_enrolment_date,child_months_enrolment,child_grant,child_years_in_programme,child_height,child_observe_attentive,...,obs_cooking_4,obs_cooking_5,obs_cooking_6,obs_heating_1,obs_heating_2,obs_heating_3,obs_heating_4,obs_heating_5,obs_heating_6,obs_heating_7
0,ID_0I0999N6S,2021.0,2021-09-20,57.000000,NaN,NaN,Yes,2nd year in programme,108.0,Almost always,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ID_GQ6ONJ4FP,2021.0,2021-10-21,54.000000,2021-01-10,9.0,Yes,1st year in the programme,105.0,Almost always,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ID_YZ76CVRW3,2021.0,2021-05-17,57.000000,NaN,NaN,Yes,NaN,101.5,Often,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ID_BNINCRXH8,2022.0,2022-09-09,59.334702,NaN,NaN,NaN,3rd year in programme,NaN,Almost always,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,ID_1U7GDTLRI,2021.0,2021-10-12,54.000000,2021-01-15,8.0,Yes,1st year in the programme,103.5,Often,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# Preview submission file
ss.head()

,child_id,target,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15
0,ID_0I0999N6S,0,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature
1,ID_GQ6ONJ4FP,0,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature
2,ID_YZ76CVRW3,0,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature
3,ID_BNINCRXH8,0,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature
4,ID_1U7GDTLRI,0,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature


In [7]:
# For a simple model model we will use only numerical columns
y = train.target

num_cols = list(set(test.select_dtypes('number')).intersection(train.select_dtypes('number')))
X = train[num_cols]
X = X.fillna(0)

# Split data into training and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1234)

# Train a model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Check score
mean_squared_error(y_test, y_pred, squared=False)

11.420457447004793

In [8]:
# Preview overall top 10 features
eli5.show_weights(model, feature_names=num_cols)

In [9]:
# Select a single/specific prediction
sample = X_train.iloc[[1]]
sample

,count_staff_paid_assistants,pri_difficult_learn,count_practitioners_age_4,obs_cooking_4,count_staff_contract_permanent,id_team,teacher_selfcare_total,latitude,count_practitioners_age_0,language_match,...,count_staff_paid_managers,child_attendance,count_staff_time,count_practitioners_age_1,count_staff_qual_nqf6_9,obs_heating_2,count_register_year_grader,obs_heating_3,obs_lighting_5,pqa_score_assessment
5273,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
# Preview top 5 predictors for the single/specific prediction
eli5.show_prediction(model, X_test.iloc[1], feature_names=num_cols, show_feature_values=True, top = 5)

In [11]:
# We can get the top 15 features as a pandas dataframe
eli5.explain_prediction_df(estimator=model, doc=X_test.iloc[1], top = 15)

,target,feature,weight,value
0,y,data_year,1910.194771,2019.000000
1,y,child_age,52.741001,63.770493
2,y,teacher_emotional_total,12.391657,10.000000
3,y,child_observe_total,9.465079,6.000000
4,y,ses_proxy,1.019442,1.000000
5,y,id_dc_n,0.969180,695.000000
6,y,child_zha,0.300672,0.196988
7,y,id_mn_n,-0.165037,366.000000
8,y,id_prov_n,-0.850246,1408.000000
9,y,child_height,-0.881085,112.000000


In [12]:
# Make predictions on the test set
preds = model.predict(test[num_cols].fillna(0))
preds

array([52.60154428, 49.51569132, 40.93002631, ..., 49.51611632,
       31.98274979, 41.55571452])

In [13]:
# Get the top five features for each predictions
top_predictors = []
for i in range(test.shape[0]):
  predictors = eli5.explain_prediction_df(estimator=model, doc=test[num_cols].fillna(0).iloc[i], top = 15)
  top_predictors.append(predictors.feature.tolist())

In [14]:
# Prepare your sub file to look like the provided sample submission
ss.head()

,child_id,target,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15
0,ID_0I0999N6S,0,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature
1,ID_GQ6ONJ4FP,0,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature
2,ID_YZ76CVRW3,0,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature
3,ID_BNINCRXH8,0,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature
4,ID_1U7GDTLRI,0,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature


In [15]:
# Submission file preparation
predictors = pd.DataFrame(top_predictors, columns = ['feature_' + str(i) for i in range(1,16)])
predictors['target'] = preds
predictors['child_id'] = test.child_id
predictors = predictors[ss.columns.tolist()]
predictors.head()

,child_id,target,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15
0,ID_0I0999N6S,52.601544,data_year,child_age,count_register_gender,count_register_race_african,count_staff_salary,count_staff_gender_female,child_observe_total,teacher_emotional_total,count_staff_gender,language_match,count_register_gender_male,count_register_gender_female,count_register_race,count_staff_salary_paid,<BIAS>
1,ID_GQ6ONJ4FP,49.515691,data_year,count_staff_salary,child_age,count_staff_gender_female,count_register_gender,count_register_race_african,child_observe_total,teacher_emotional_total,count_staff_contract,language_match,count_staff_gender,count_register_gender_male,count_register_race,count_staff_salary_paid,<BIAS>
2,ID_YZ76CVRW3,40.930026,data_year,child_age,count_staff_gender_female,count_staff_salary,pri_fees_amount,child_observe_total,count_register_gender,count_register_race_african,count_register_gender_female,longitude,count_register_race,language_match,pri_fees_amount_pv,count_staff_salary_paid,<BIAS>
3,ID_BNINCRXH8,62.392599,data_year,count_register_gender,count_register_race_african,count_staff_gender_female,count_staff_salary,child_age,child_observe_total,count_staff_time,count_staff_contract,count_staff_gender,count_register_gender_male,count_register_gender_female,count_register_race,count_staff_salary_paid,<BIAS>
4,ID_1U7GDTLRI,46.250745,data_year,child_age,count_staff_gender_female,count_register_gender,count_staff_salary,count_register_race_coloured,child_observe_total,count_staff_contract,count_staff_gender,language_match,count_register_gender_female,count_register_gender_male,count_register_race,count_staff_salary_paid,<BIAS>


In [16]:
# Create csv to upload to Zindi for scoring
predictors.to_csv('BaselineSubmission.csv', index = False)

##Note
- Please note that you are not restricted to use eli5 for model interpretability. You can use any other open source libraries like:
      - SHAP
      - LIME
      - Yellowbrick
      - Lucid
      - ... and any other